In [75]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wave
import numpy.fft
from IPython.display import Audio

In [76]:
audio_base_path = 'audio/'
audio_file_name = 'hawking01.wav'

In [94]:
# fs is sample rate, audio_data is data from a .wav file
fs, audio_data = wave.read(audio_base_path + audio_file_name)

# audio length in seconds
length = audio_data.shape[0] / fs

print("Audio length:", round(length, 2), "seconds")
print("Sample rate:", fs, "Hz")

Audio length: 21.18 seconds
Sample rate: 11025 Hz


In [95]:
Audio(audio_data, rate=fs)